In [2]:
import time
import os
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt

from siriuspy.search import IDSearch
from siriuspy.devices import IVU, SOFB, HLFOFB, IDFF, Tune, DVFImgProc

import epics
from mathphys.functions import save, load

timeout = 3

# Functions definition

In [2]:
def initialize_ivu(beamline):
    # Search ID
    devnameivu = IDSearch.conv_beamline_2_idname(beamline=beamline)
    ivu = IVU(devname=devnameivu)

    # Disable beamline control
    ivu.cmd_beamline_ctrl_disable()
    print('beamline control: ', ivu.is_beamline_ctrl_enabled)

    # Disable center mode and pitch mode
    ivu.set_center_mode(False)
    ivu.set_pitch_mode(False)
    time.sleep(0.5)
    print('center mode: ', ivu.center_mode_status)
    print('pitch mode: ', ivu.pitch_mode_status)

    # Set gap speed
    ivu.set_gap_speed(0.1)
    time.sleep(0.5)
    print('gap speed: {:.3f} mm/s'.format(ivu.gap_speed))

    # Set gap to parked condition
    ivu.set_gap(ivu.kparameter_parked)
    time.sleep(0.5)
    print('gap: {:.3f} mm'.format(ivu.gap))

    return ivu

def move_ivu_gap(ivu:IVU, gap, timeout, verbose=False):
    ivu.set_gap(gap)
    time.sleep(0.5)
    print('Gap-RB {:.3f} mm'.format(ivu.gap)) if verbose else 0
    if ivu.cmd_move_gap_start(timeout):
        time.sleep(0.5)
        print('Undulator is moving...') if verbose else 0
        while ivu.is_moving:
            time.sleep(0.1)
            print('Current gap {:.3f} mm.'.format(ivu.gap_mon), end='\r') if verbose else 0
        print('Gap {:.3f} mm reached.'.format(ivu.gap)) if verbose else 0
        return True
    else:
        print('Error while cmd_move_start.')
        return False

def move_ivu_gap_robust(ivu:IVU, gap, timeout, maxiter=3, verbose=False):
    sucess = move_ivu_gap(ivu, gap=gap, timeout=timeout, verbose=verbose)
    i=0
    while not sucess and i<maxiter:
        i += 1
        print('Trying {:0f}/{:.0f}'.format(i, maxiter)) if verbose else 0
        ivu.cmd_reset(timeout=timeout)
        time.sleep(0.5)
        sucess = move_ivu_gap(ivu, gap=gap, timeout=timeout, verbose=verbose)
    if sucess:
        print('Movimentation done!\n')
        return True
    else:
        print('Error while moving.\n')
        return False

def get_CAX_data(tune, imgproc, nr_acq=10, acq_sleep=0.5):
        """Get data.

        Returns:
            data: Numpy 1d array
        """
        sigmax = np.zeros(nr_acq)
        sigmay = np.zeros(nr_acq)
        sigma1 = np.zeros(nr_acq)
        sigma2 = np.zeros(nr_acq)
        angle = np.zeros(nr_acq)
        tunex = np.zeros(nr_acq)
        tuney = np.zeros(nr_acq)
        images = np.zeros((nr_acq, imgproc.image_sizey, imgproc.image_sizex))
        for i in range(nr_acq):
            sigmax[i] = imgproc.roix_fit_sigma
            sigmay[i] = imgproc.roiy_fit_sigma
            sigma1[i] = imgproc.fit_sigma1
            sigma2[i] = imgproc.fit_sigma2
            angle[i] = imgproc.fit_angle
            tunex[i] = tune.tunex
            tuney[i] = tune.tuney
            images[i] = imgproc.image
            time.sleep(acq_sleep)

        img_params = np.zeros((7, nr_acq))
        img_params[0, :] = sigmax
        img_params[1, :] = sigmay
        img_params[2, :] = sigma1
        img_params[3, :] = sigma2
        img_params[4, :] = angle
        img_params[5, :] = tunex
        img_params[6, :] = tuney
        return img_params, images


## Search devnames

In [4]:
devnameivu_ema = IDSearch.conv_beamline_2_idname(beamline='EMA')
devnameivu_paineira = IDSearch.conv_beamline_2_idname(beamline='PAINEIRA')

## Initialize devices

In [ ]:
ivu = initialize_ivu(beamline='EMA')
ivu.set_gap_speed(0.5)
time.sleep(0.5)
print(ivu.gap_speed)

In [ ]:
tune = Tune(Tune.DEVICES.SI)
imgproc = DVFImgProc(DVFImgProc.DEVICES.CAX_DVF2)

imgproc.wait_for_connection(timeout=timeout)
print('IMG Proc connected: ', imgproc.connected)

tune.wait_for_connection(timeout=timeout)
print('Tune connected: ', tune.connected)


In [ ]:
sofb = SOFB(SOFB.DEVICES.SI)
fofb = HLFOFB(HLFOFB.DEVICES.SI)

### Make sure SOFB and FOFB loops are opened

In [ ]:
sofb.cmd_turn_off_autocorr()
sofb.cmd_change_opmode_to_sloworb(timeout=timeout)
fofb.cmd_turn_off_loop_state()

# Meas orbit distortions

In [ ]:
gaps = np.arange(24, 3, -1)
print(gaps)

In [ ]:
data = dict()
data['gap'] = list()
data['timestamps'] = list()
data['orbx'] = list()
data['orby'] = list()
data['gap_speed'] = list()
data['sigmax'] = list()
data['sigmay'] = list()
data['sigma1'] = list()
data['sigma2'] = list()
data['angle'] = list()
data['tunex'] = list()
data['tuney'] = list()
data['CAX_images'] = list()
t0 = time.time()
for i, gap in enumerate(gaps):

    # Move gap
    sucess = move_ivu_gap_robust(ivu, gap=gap, timeout=timeout, verbose=True)
    
    # Verify if undulator moved
    if not sucess:
        break

    # Wait orbit distortion to become stationary
    time.sleep(3)
    
    # Record orbit and other info
    data['gap'].append(gap)
    data['timestamps'].append(time.time())
    data['orbx'].append(sofb.orbx)
    data['orby'].append(sofb.orby)
    data['gap_speed'].append(ivu.gap_speed)
    data['beamline'] = IDSearch.conv_idname_2_beamline(ivu.devname)

    # Record CAX img params and tune
    caxdata, images = get_CAX_data(tune, imgproc, nr_acq=10, acq_sleep=0.5)
    data['sigmax'].append(caxdata[0, :])
    data['sigmay'].append(caxdata[1, :])
    data['sigma1'].append(caxdata[2, :])
    data['sigma2'].append(caxdata[3, :])
    data['angle'].append(caxdata[4, :])
    data['tunex'].append(caxdata[5, :])
    data['tuney'].append(caxdata[6, :])
    data['CAX_images'].append(images)

print(f'Done! ETA: {time.time()-t0:.3f}s')
save(data, 'orbit_distortions_CAX_IVU_{}_iter0.pickle'.format(IDSearch.conv_idname_2_beamline(ivu.devname)))

# Meas local correctors matrix response

In [ ]:
# Open ivu gap
move_ivu_gap_robust(ivu, gap=24, timeout=timeout, verbose=True)

In [ ]:
idff = IDFF(ivu.devname)

# Configure SOFB and FOFB
sofb.cmd_turn_off_autocorr()
sofb.cmd_change_opmode_to_sloworb(timeout=timeout)
fofb.cmd_turn_off_loop_state()

sofb.correct_orbit_manually(nr_iters=5, residue=2)
time.sleep(2)



In [ ]:
ivu = IVU(devnameivu_ema)
idff = IDFF(ivu.devname)

dcurr = 0.1
corrs = list()
corrs.extend(idff.chdevs)
corrs.extend(idff.cvdevs)

data = dict()
data['delta_current'] = 2*dcurr
for i, corr in enumerate(corrs):
    data[corr.devname] = dict()
    corr.cmd_turn_on(timeout=timeout)
    corr.current = 0
    time.sleep(1)
    print('Current {:.2f} A, corrector: '.format(corr.current_mon), corr.devname)

t0 = time.time()
for i, corr in enumerate(corrs):
    sofb.correct_orbit_manually(nr_iters=5, residue=2)
    time.sleep(3)
    orbx0 = sofb.orbx
    orby0 = sofb.orby

    print('Measuring corrector: ', corr.devname)
    
    corr.current = dcurr
    time.sleep(3)
    data[corr.devname]['pos'] = dict()
    data[corr.devname]['pos']['timestamps'] = time.time()
    data[corr.devname]['pos']['dorbx'] = sofb.orbx - orbx0
    data[corr.devname]['pos']['dorby'] = sofb.orby - orby0

    corr.current = -dcurr
    time.sleep(3)
    data[corr.devname]['neg'] = dict()
    data[corr.devname]['neg']['timestamps'] = time.time()
    data[corr.devname]['neg']['dorbx'] = sofb.orbx - orbx0
    data[corr.devname]['neg']['dorby'] = sofb.orby - orby0

    corr.current = 0
    

print(f'Done! ETA: {time.time()-t0:.3f}s')
save(data, 'orbit_distortions_corrs_IVU_{}_iter0.pickle'.format(IDSearch.conv_idname_2_beamline(ivu.devname)))
